#In this notebook we load our fine-tuned Distilbert models and evaluate them on the validation and test data sets

# Install

In [ ]:
!pip install -U -q datasets transformers torchinfo

     |████████████████████████████████| 441 kB 15.1 MB/s 
     |████████████████████████████████| 5.3 MB 64.1 MB/s 
     |████████████████████████████████| 212 kB 59.2 MB/s 
     |████████████████████████████████| 115 kB 66.5 MB/s 
     |████████████████████████████████| 163 kB 68.3 MB/s 
     |████████████████████████████████| 127 kB 70.9 MB/s 
     |████████████████████████████████| 7.6 MB 53.1 MB/s 
     |████████████████████████████████| 115 kB 35.3 MB/s 


# Imports

In [ ]:
from datasets import Dataset, DatasetDict, ClassLabel, Value, load_dataset, load_metric
from huggingface_hub import notebook_login
from torchinfo import summary
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
import pandas as pd
from sklearn.metrics import classification_report

# Data loading

In [ ]:
# login to HF
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [ ]:
#load dataset from our HF dataset repo
ds = load_dataset('sara-nabhani/lfd-proj',
    'csv', 
    data_files={'train': 'train.csv', 'val': 'val.csv', 'test': 'test.csv'}
)

cl = ClassLabel(names=list(ds['train'].unique('label')))
ds = ds.cast_column('label', cl)

ds

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/sara-nabhani___csv/sara-nabhani--lfd-proj-a35fbd5b9bbbc3d0/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Casting the dataset:   0%|          | 0/2 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['tweet', 'label'],
        num_rows: 12240
    })
    val: Dataset({
        features: ['tweet', 'label'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['tweet', 'label'],
        num_rows: 860
    })
})

# distilbert-base-cased-finetuned-0 Model Evaluation

In [ ]:
model_id = 'sara-nabhani/distilbert-base-cased-finetuned-0'
# load tokenizerand tokenize the dataset
tokenizer = AutoTokenizer.from_pretrained(model_id)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def tokenize_function(examples):
    return tokenizer(examples['tweet'], padding='max_length', truncation=True)

tokenized_ds = ds.map(tokenize_function, batched=True)

Downloading:   0%|          | 0.00/359 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/669k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

  0%|          | 0/13 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
metric = load_metric("accuracy")
metric1= load_metric("f1")
# evaluation function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"acc": metric.compute(predictions=predictions, references=labels), "f1":metric1.compute(predictions=predictions, references=labels, average='macro')}
#load model
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2)
trainer = Trainer(
  model=model,
  train_dataset=tokenized_ds['train'],
  eval_dataset=tokenized_ds['val'],
  compute_metrics=compute_metrics,
  tokenizer=tokenizer,
  data_collator=data_collator
  )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  """Entry point for launching an IPython kernel.


Downloading:   0%|          | 0.00/736 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/263M [00:00<?, ?B/s]

##evaluation

In [ ]:

print("Prediction on validation data:")
print(trainer.predict(tokenized_ds['val'])[-1])

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: tweet. If tweet are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1000
  Batch size = 8
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Prediction on validation data:


{'test_loss': 0.4627390205860138, 'test_acc': {'accuracy': 0.794}, 'test_f1': {'f1': 0.7736263736263737}, 'test_runtime': 17.7367, 'test_samples_per_second': 56.38, 'test_steps_per_second': 7.048}


In [ ]:
print("Prediction on test data:")
print(trainer.predict(tokenized_ds['test'])[-1])

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: tweet. If tweet are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 860
  Batch size = 8


Prediction on test data:


{'test_loss': 0.38811683654785156, 'test_acc': {'accuracy': 0.8441860465116279}, 'test_f1': {'f1': 0.7983552055993002}, 'test_runtime': 14.0462, 'test_samples_per_second': 61.227, 'test_steps_per_second': 7.689}


## saving to DF

In [ ]:
val_predictions = np.argmax(trainer.predict(tokenized_ds['val'])[0], axis=-1)
test_predictions = np.argmax(trainer.predict(tokenized_ds['test'])[0], axis=-1)

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: tweet. If tweet are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1000
  Batch size = 8


The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: tweet. If tweet are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 860
  Batch size = 8


In [ ]:
val_data_frame=pd.DataFrame({'tweet': ds['val']['tweet'], 'true_label': ds['val']['label'], 'pred_label': val_predictions})
test_data_frame=pd.DataFrame({'tweet': ds['test']['tweet'], 'true_label': ds['test']['label'], 'pred_label': test_predictions})

In [ ]:
print("Validation classification report:")
print(classification_report(val_data_frame['true_label'], val_data_frame['pred_label']))

Validation classification report:
              precision    recall  f1-score   support

           0       0.71      0.70      0.71       352
           1       0.84      0.84      0.84       648

    accuracy                           0.79      1000
   macro avg       0.77      0.77      0.77      1000
weighted avg       0.79      0.79      0.79      1000



In [ ]:
print("Test classification report:")
print(classification_report(test_data_frame['true_label'], test_data_frame['pred_label']))

Test classification report:
              precision    recall  f1-score   support

           0       0.75      0.66      0.70       240
           1       0.87      0.92      0.89       620

    accuracy                           0.84       860
   macro avg       0.81      0.79      0.80       860
weighted avg       0.84      0.84      0.84       860



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
test_data_frame.to_csv('/content/gdrive/MyDrive/lfd-proj/models-output/distilbert-base-cased-finetuned-0.csv', index=False)
val_data_frame.to_csv('/content/gdrive/MyDrive/lfd-proj/val-models-output/distilbert-base-cased-finetuned-0.csv', index=False)

# distilbert-base-cased-finetuned-1 Model Evaluation

In [ ]:
model_id = 'sara-nabhani/distilbert-base-cased-finetuned-1'
# load tokenizerand tokenize the dataset
tokenizer = AutoTokenizer.from_pretrained(model_id)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def tokenize_function(examples):
    return tokenizer(examples['tweet'], padding='max_length', truncation=True)

tokenized_ds = ds.map(tokenize_function, batched=True)

print(len(tokenized_ds['train']['input_ids'][0]))
metric = load_metric("accuracy")
metric1= load_metric("f1")
# evaluation function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"acc": metric.compute(predictions=predictions, references=labels), "f1":metric1.compute(predictions=predictions, references=labels, average='macro')}
#load model
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2)
trainer = Trainer(
  model=model,
  train_dataset=tokenized_ds['train'],
  eval_dataset=tokenized_ds['val'],
  compute_metrics=compute_metrics,
  tokenizer=tokenizer,
  data_collator=data_collator
  )
#evaluation
print("Prediction on validation data:")
print(trainer.predict(tokenized_ds['val'])[-1])
print("Prediction on test data:")
print(trainer.predict(tokenized_ds['test'])[-1])
#saving to DF
val_predictions = np.argmax(trainer.predict(tokenized_ds['val'])[0], axis=-1)
test_predictions = np.argmax(trainer.predict(tokenized_ds['test'])[0], axis=-1)
val_data_frame=pd.DataFrame({'tweet': ds['val']['tweet'], 'true_label': ds['val']['label'], 'pred_label': val_predictions})
test_data_frame=pd.DataFrame({'tweet': ds['test']['tweet'], 'true_label': ds['test']['label'], 'pred_label': test_predictions})
print("Validation classification report:")
print(classification_report(val_data_frame['true_label'], val_data_frame['pred_label']))
print("Test classification report:")
print(classification_report(test_data_frame['true_label'], test_data_frame['pred_label']))
test_data_frame.to_csv('/content/gdrive/MyDrive/lfd-proj/models-output/distilbert-base-cased-finetuned-1.csv', index=False)
val_data_frame.to_csv('/content/gdrive/MyDrive/lfd-proj/val-models-output/distilbert-base-cased-finetuned-1.csv', index=False)

Downloading:   0%|          | 0.00/359 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/669k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--sara-nabhani--distilbert-base-cased-finetuned-1/snapshots/8cbd60e92e91e5be489ca62c67d980937932ba29/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--sara-nabhani--distilbert-base-cased-finetuned-1/snapshots/8cbd60e92e91e5be489ca62c67d980937932ba29/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--sara-nabhani--distilbert-base-cased-finetuned-1/snapshots/8cbd60e92e91e5be489ca62c67d980937932ba29/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--sara-nabhani--distilbert-base-cased-finetuned-1/snapshots/8cbd60e92e91e5be489ca62c67d980937932ba29/tokenizer_config.json


...

{'test_loss': 0.46126624941825867, 'test_acc': {'accuracy': 0.791}, 'test_f1': {'f1': 0.7669904644470928}, 'test_runtime': 17.5002, 'test_samples_per_second': 57.142, 'test_steps_per_second': 7.143}
Prediction on test data:


The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: tweet. If tweet are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1000
  Batch size = 8


{'test_loss': 0.38841426372528076, 'test_acc': {'accuracy': 0.836046511627907}, 'test_f1': {'f1': 0.7856502701909632}, 'test_runtime': 15.1194, 'test_samples_per_second': 56.881, 'test_steps_per_second': 7.143}


The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: tweet. If tweet are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 860
  Batch size = 8


Validation classification report:
              precision    recall  f1-score   support

           0       0.72      0.67      0.69       352
           1       0.83      0.86      0.84       648

    accuracy                           0.79      1000
   macro avg       0.77      0.76      0.77      1000
weighted avg       0.79      0.79      0.79      1000

Test classification report:
              precision    recall  f1-score   support

           0       0.74      0.63      0.68       240
           1       0.86      0.92      0.89       620

    accuracy                           0.84       860
   macro avg       0.80      0.77      0.79       860
weighted avg       0.83      0.84      0.83       860



# distilbert-base-cased-finetuned-2 Model Evaluation

In [ ]:
model_id = 'sara-nabhani/distilbert-base-cased-finetuned-2'
# load tokenizerand tokenize the dataset
tokenizer = AutoTokenizer.from_pretrained(model_id)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def tokenize_function(examples):
    return tokenizer(examples['tweet'], padding='max_length', truncation=True, max_length=100)

tokenized_ds = ds.map(tokenize_function, batched=True)

print(len(tokenized_ds['train']['input_ids'][0]))
metric = load_metric("accuracy")
metric1= load_metric("f1")
# evaluation function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"acc": metric.compute(predictions=predictions, references=labels), "f1":metric1.compute(predictions=predictions, references=labels, average='macro')}
#load model
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2)
trainer = Trainer(
  model=model,
  train_dataset=tokenized_ds['train'],
  eval_dataset=tokenized_ds['val'],
  compute_metrics=compute_metrics,
  tokenizer=tokenizer,
  data_collator=data_collator
  )
#evaluation
print("Prediction on validation data:")
print(trainer.predict(tokenized_ds['val'])[-1])
print("Prediction on test data:")
print(trainer.predict(tokenized_ds['test'])[-1])
#saving to DF
val_predictions = np.argmax(trainer.predict(tokenized_ds['val'])[0], axis=-1)
test_predictions = np.argmax(trainer.predict(tokenized_ds['test'])[0], axis=-1)
val_data_frame=pd.DataFrame({'tweet': ds['val']['tweet'], 'true_label': ds['val']['label'], 'pred_label': val_predictions})
test_data_frame=pd.DataFrame({'tweet': ds['test']['tweet'], 'true_label': ds['test']['label'], 'pred_label': test_predictions})
print("Validation classification report:")
print(classification_report(val_data_frame['true_label'], val_data_frame['pred_label']))
print("Test classification report:")
print(classification_report(test_data_frame['true_label'], test_data_frame['pred_label']))
test_data_frame.to_csv('/content/gdrive/MyDrive/lfd-proj/models-output/distilbert-base-cased-finetuned-2.csv', index=False)
val_data_frame.to_csv('/content/gdrive/MyDrive/lfd-proj/val-models-output/distilbert-base-cased-finetuned-2.csv', index=False)

Downloading:   0%|          | 0.00/359 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/669k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--sara-nabhani--distilbert-base-cased-finetuned-2/snapshots/ed1c985b7586037bdcca2630aaa1f384786c8e80/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--sara-nabhani--distilbert-base-cased-finetuned-2/snapshots/ed1c985b7586037bdcca2630aaa1f384786c8e80/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--sara-nabhani--distilbert-base-cased-finetuned-2/snapshots/ed1c985b7586037bdcca2630aaa1f384786c8e80/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--sara-nabhani--distilbert-base-cased-finetuned-2/snapshots/ed1c985b7586037bdcca2630aaa1f384786c8e80/tokenizer_config.json


...

{'test_loss': 0.45667147636413574, 'test_acc': {'accuracy': 0.802}, 'test_f1': {'f1': 0.7809492200464654}, 'test_runtime': 3.3719, 'test_samples_per_second': 296.566, 'test_steps_per_second': 37.071}
Prediction on test data:


The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: tweet. If tweet are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1000
  Batch size = 8


{'test_loss': 0.38600823283195496, 'test_acc': {'accuracy': 0.8395348837209302}, 'test_f1': {'f1': 0.7923359580052494}, 'test_runtime': 2.6671, 'test_samples_per_second': 322.452, 'test_steps_per_second': 40.494}


The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: tweet. If tweet are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 860
  Batch size = 8


Validation classification report:
              precision    recall  f1-score   support

           0       0.73      0.70      0.71       352
           1       0.84      0.86      0.85       648

    accuracy                           0.80      1000
   macro avg       0.78      0.78      0.78      1000
weighted avg       0.80      0.80      0.80      1000

Test classification report:
              precision    recall  f1-score   support

           0       0.74      0.65      0.69       240
           1       0.87      0.91      0.89       620

    accuracy                           0.84       860
   macro avg       0.81      0.78      0.79       860
weighted avg       0.84      0.84      0.84       860



# distilbert-base-cased-finetuned-3 Model Evaluation

In [ ]:
model_id = 'sara-nabhani/distilbert-base-cased-finetuned-3'
# load tokenizerand tokenize the dataset
tokenizer = AutoTokenizer.from_pretrained(model_id)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def tokenize_function(examples):
    return tokenizer(examples['tweet'], padding='max_length', truncation=True, max_length=200)

tokenized_ds = ds.map(tokenize_function, batched=True)

print(len(tokenized_ds['train']['input_ids'][0]))
metric = load_metric("accuracy")
metric1= load_metric("f1")
# evaluation function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"acc": metric.compute(predictions=predictions, references=labels), "f1":metric1.compute(predictions=predictions, references=labels, average='macro')}
# load model
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2)
trainer = Trainer(
  model=model,
  train_dataset=tokenized_ds['train'],
  eval_dataset=tokenized_ds['val'],
  compute_metrics=compute_metrics,
  tokenizer=tokenizer,
  data_collator=data_collator
  )
# evaluation
print("Prediction on validation data:")
print(trainer.predict(tokenized_ds['val'])[-1])
print("Prediction on test data:")
print(trainer.predict(tokenized_ds['test'])[-1])
# saving to DF
val_predictions = np.argmax(trainer.predict(tokenized_ds['val'])[0], axis=-1)
test_predictions = np.argmax(trainer.predict(tokenized_ds['test'])[0], axis=-1)
val_data_frame=pd.DataFrame({'tweet': ds['val']['tweet'], 'true_label': ds['val']['label'], 'pred_label': val_predictions})
test_data_frame=pd.DataFrame({'tweet': ds['test']['tweet'], 'true_label': ds['test']['label'], 'pred_label': test_predictions})
print("Validation classification report:")
print(classification_report(val_data_frame['true_label'], val_data_frame['pred_label']))
print("Test classification report:")
print(classification_report(test_data_frame['true_label'], test_data_frame['pred_label']))
test_data_frame.to_csv('/content/gdrive/MyDrive/lfd-proj/models-output/distilbert-base-cased-finetuned-3.csv', index=False)
val_data_frame.to_csv('/content/gdrive/MyDrive/lfd-proj/val-models-output/distilbert-base-cased-finetuned-3.csv', index=False)

Downloading:   0%|          | 0.00/359 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/669k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--sara-nabhani--distilbert-base-cased-finetuned-3/snapshots/c7465a26d634da30e742c623c7e6d639319762e3/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--sara-nabhani--distilbert-base-cased-finetuned-3/snapshots/c7465a26d634da30e742c623c7e6d639319762e3/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--sara-nabhani--distilbert-base-cased-finetuned-3/snapshots/c7465a26d634da30e742c623c7e6d639319762e3/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--sara-nabhani--distilbert-base-cased-finetuned-3/snapshots/c7465a26d634da30e742c623c7e6d639319762e3/tokenizer_config.json


...

{'test_loss': 0.4575091600418091, 'test_acc': {'accuracy': 0.798}, 'test_f1': {'f1': 0.7755914609218081}, 'test_runtime': 6.2941, 'test_samples_per_second': 158.878, 'test_steps_per_second': 19.86}
Prediction on test data:


The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: tweet. If tweet are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1000
  Batch size = 8


{'test_loss': 0.38745373487472534, 'test_acc': {'accuracy': 0.8465116279069768}, 'test_f1': {'f1': 0.7990355405434113}, 'test_runtime': 5.4451, 'test_samples_per_second': 157.941, 'test_steps_per_second': 19.834}


The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: tweet. If tweet are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 860
  Batch size = 8


Validation classification report:
              precision    recall  f1-score   support

           0       0.73      0.68      0.70       352
           1       0.83      0.86      0.85       648

    accuracy                           0.80      1000
   macro avg       0.78      0.77      0.78      1000
weighted avg       0.80      0.80      0.80      1000

Test classification report:
              precision    recall  f1-score   support

           0       0.77      0.65      0.70       240
           1       0.87      0.92      0.90       620

    accuracy                           0.85       860
   macro avg       0.82      0.79      0.80       860
weighted avg       0.84      0.85      0.84       860

